In [5]:
import torch
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import load_image
from transformers import CLIPVisionModelWithProjection

# ==========================================
# 1. Image Encoder 교체 (핵심 수정 부분)
# ==========================================
# IP-Adapter-Plus 계열(Face 포함)은 SDXL 기본 인코더(BigG, 1664)가 아니라
# CLIP-ViT-H-14(1024/1280)를 사용해야 합니다.
image_encoder = CLIPVisionModelWithProjection.from_pretrained(
    "h94/IP-Adapter", 
    subfolder="models/image_encoder",  # IP-Adapter 제작자가 제공하는 전용 인코더
    torch_dtype=torch.float16
).to("cuda")

# ==========================================
# 2. 파이프라인 설정
# ==========================================
pipeline = AutoPipelineForImage2Image.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    image_encoder=image_encoder,  # <--- 교체한 인코더를 여기에 주입!
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

# ==========================================
# 3. IP-Adapter 로드 (Plus Face 버전 예시)
# ==========================================
# 만약 "Face" 특성을 잘 살리고 싶다면 파일명을 정확히 지정해야 합니다.
pipeline.load_ip_adapter(
    "h94/IP-Adapter", 
    subfolder="sdxl_models", 
    weight_name="ip-adapter-plus-face_sdxl_vit-h.bin" # 혹은 사용하려는 정확한 bin 파일명
)

# ==========================================
# 4. 이미지 생성 실행 (기존과 동일)
# ==========================================
generator = torch.Generator(device="cuda").manual_seed(42)
init_image = load_image("image.png") # 테스트용

image = pipeline(
# 프롬프트 핵심: 광고 포스터 스타일 명시, 큰 글씨 내용 정확히 기재
    prompt="A bright retail advertisement poster showing the object. Big bold red text running across says 'SPECIAL SALE 50% OFF'. Promotional signage, supermarket background, high quality, clear text.",
    
    # 네거티브 프롬프트 핵심: 글자가 뭉개지거나 이상하게 나오는 것을 방지
    negative_prompt="low quality, blurred, ugly text, unreadable text, watermark, messy fonts",
    
    image=init_image,            
    ip_adapter_image=init_image, 
    strength=0.75,
    guidance_scale=7.5,
    num_inference_steps=30,
    generator=generator
).images[0]

image.save("nano_banana_fixed.png")

100%|██████████| 22/22 [00:04<00:00,  4.89it/s]


In [6]:
import os
import requests
from PIL import Image, ImageDraw, ImageFont

# ==========================================
# 0. 한글 폰트 준비 (나눔고딕 다운로드)
# ==========================================
def download_font():
    font_url = "https://github.com/google/fonts/raw/main/ofl/nanumgothic/NanumGothic-Bold.ttf"
    font_path = "NanumGothic-Bold.ttf"
    if not os.path.exists(font_path):
        print("한글 폰트 다운로드 중...")
        r = requests.get(font_url)
        with open(font_path, 'wb') as f:
            f.write(r.content)
    return font_path

font_path = download_font()

# ==========================================
# 1. SDXL로 이미지 생성 (글자 없이 그림만)
# ==========================================
# 프롬프트에서 텍스트 관련 내용은 뺍니다. 공간만 확보합니다.
generator = torch.Generator(device="cuda").manual_seed(42)

# 예: "여백이 있는 광고 포스터 배경"
image = pipeline(
    prompt="product photography, minimalist advertisement background, soft lighting, high quality, empty space at the top",
    negative_prompt="text, watermark, writing, low quality",
    image=init_image,            
    ip_adapter_image=init_image, 
    strength=0.7,   # 원본 유지 강도 조절
    guidance_scale=7.5,
    num_inference_steps=30,
    generator=generator
).images[0]

# ==========================================
# 2. Python으로 한글 '특가 세일' 합성
# ==========================================
draw = ImageDraw.Draw(image)

# 폰트 설정 (사이즈 100)
try:
    font = ImageFont.truetype(font_path, 100) 
except:
    font = ImageFont.load_default() # 만약 실패하면 기본 폰트(한글 안나옴)

text = "특가 세일 50%"

# 글자 위치 계산 (가운데 정렬)
# textbbox는 (left, top, right, bottom) 반환
bbox = draw.textbbox((0, 0), text, font=font)
text_width = bbox[2] - bbox[0]
text_height = bbox[3] - bbox[1]

# 이미지 중앙 상단에 배치
img_width, img_height = image.size
x = (img_width - text_width) / 2
y = 100 # 위에서 100픽셀 내려온 지점

# 글자 테두리(Shadow) 효과 (가독성을 위해 검은 그림자 추가)
draw.text((x+3, y+3), text, font=font, fill="black")

# 실제 글자 (빨간색)
draw.text((x, y), text, font=font, fill="red")

# 저장
image.save("nano_banana_korean.png")
print("한글 포스터 생성 완료: nano_banana_korean.png")

한글 폰트 다운로드 중...


100%|██████████| 21/21 [00:04<00:00,  4.88it/s]


한글 포스터 생성 완료: nano_banana_korean.png
